# PCA and shit 

In [5]:
import h5py 
import numpy as np
import pandas as pd
import seaborn as sns

import os 
import random
random.seed(42)

from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA


In [16]:
data_parent_dir = "./data_fixed_crop_w_mask"
precomputed_path = "features_matrix.csv"

In [22]:
precomputed_df = pd.read_csv(precomputed_path)
feature_names = list(precomputed_df)[1:]
precomputed_fts = precomputed_df.values[:,1:]
precomputed_fts =np.array(precomputed_fts,dtype=np.float32)
print(precomputed_fts[0])

[ 9.8299998e-01  1.2056100e+05  5.4922956e-01  1.4699945e-01
  2.7999999e-03  1.6000000e-03  5.3562564e-01  9.9140000e-01
  9.9940002e-01  1.9901167e+02  1.4164154e+02  8.8326851e+01
 -1.4469041e+00  2.9172947e+03  7.8043795e-01  8.2708019e-01
  1.2968200e+01  9.7119999e-01]


We're going to do an initial zero-centering

In [28]:
precomputed_fts -= np.mean(precomputed_fts, axis = 0)
precomputed_fts /= np.std(precomputed_fts, axis=0)

print(precomputed_fts.shape)
precomputed_df[feature_names]

(1599, 18)


,ASM,Area,Centroid_x,Centroid_y,Contrast,Dissimilarity,Eccentricity,Energy,Homogeneity,Max intensity,Mean intensity,Min intensity,Orientation,Perimeter,Relative intensity,Solidity,StdDev intensity,Texture correlation
0,0.9830,120561.0,0.549230,0.146999,0.0028,0.0016,0.535626,0.9914,0.9994,199.011673,141.641546,88.326848,-1.446904,2917.294731,0.780438,0.827080,12.9682,0.9712
1,0.9947,32198.0,0.645281,0.141495,0.0020,0.0008,0.316796,0.9973,0.9997,195.498054,168.820506,126.961089,-1.143702,1177.159559,0.766659,0.819976,8.4965,0.9539
2,0.9858,113289.0,0.675677,0.540258,0.0036,0.0015,0.377010,0.9929,0.9995,195.669261,162.290815,122.264591,1.330022,2614.290727,0.767330,0.853195,13.4881,0.9683
3,0.9889,89475.0,0.591458,0.605354,0.0035,0.0017,0.384364,0.9944,0.9993,189.046693,150.492213,122.754864,0.761071,2816.622725,0.741360,0.799441,11.0086,0.9576
4,0.9857,108026.0,0.485827,0.353910,0.0028,0.0011,0.596902,0.9928,0.9996,212.918288,182.445551,139.101167,-0.324896,2079.655121,0.834974,0.905294,15.2720,0.9770
5,0.9882,91758.0,0.723711,0.314123,0.0029,0.0012,0.601535,0.9941,0.9996,218.396887,179.337001,128.556420,-0.028951,2268.384089,0.856458,0.837384,13.6064,0.9685
6,0.9955,21007.0,0.513685,0.264825,0.0023,0.0008,0.464911,0.9977,0.9998,189.205222,171.598492,137.667470,0.428495,897.222438,0.741981,0.780117,7.9459,0.9414
7,0.9955,29291.0,0.508352,0.261483,0.0015,0.0006,0.664359,0.9978,0.9998,173.617021,146.107791,108.510638,-1.402067,1088.045815,0.680851,0.867085,6.7809,0.9581
8,0.9962,40488.0,0.439680,0.180701,0.0009,0.0006,0.475143,0.9981,0.9997,211.969731,158.131123,120.905386,0.208830,1121.816413,0.831254,0.894604,6.7622,0.9630
9,0.9794,74597.0,0.687757,0.811424,0.0036,0.0016,0.605269,0.9896,0.9994,195.996109,158.547833,95.369650,0.745901,1426.094588,0.768612,0.896868,15.9818,0.9764


In [24]:
pca = PCA()
pca.fit(precomputed_fts)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [25]:
print(pca.explained_variance_ratio_)  
print(pca.components_[0])

[4.4473657e-01 1.6824159e-01 9.1169290e-02 5.7367690e-02 5.6748547e-02
 5.2320685e-02 3.6198188e-02 3.1822890e-02 2.2656877e-02 1.7601196e-02
 9.3034180e-03 7.1804966e-03 2.3674807e-03 1.1376145e-03 8.7159796e-04
 2.6753976e-04 6.5508975e-06 1.8152512e-06]
[-0.33582258  0.3129383  -0.04206447 -0.01204051  0.31197217  0.3208049
  0.00518964 -0.33550972 -0.30273733  0.21556541  0.16609216  0.02282244
  0.00318173  0.30442756  0.21560022 -0.05786494  0.3394732   0.22484674]
